In [6]:
import pandas as pd
import numpy as np

In [7]:
calendario = pd.read_csv("C:\RS\España\Master Data Science\TFM\capstone-project-2\capstone-project-r2\capstone-project-r2\data\daily_calendar_with_events.csv")
precios = pd.read_csv("C:\RS\España\Master Data Science\TFM\capstone-project-2\capstone-project-r2\capstone-project-r2\data\item_prices.csv")
ventas = pd.read_csv("C:\RS\España\Master Data Science\TFM\capstone-project-2\capstone-project-r2\capstone-project-r2\data\item_sales.csv")

<>:1: SyntaxWarning: invalid escape sequence '\R'
<>:2: SyntaxWarning: invalid escape sequence '\R'
<>:3: SyntaxWarning: invalid escape sequence '\R'
<>:1: SyntaxWarning: invalid escape sequence '\R'
<>:2: SyntaxWarning: invalid escape sequence '\R'
<>:3: SyntaxWarning: invalid escape sequence '\R'
C:\Users\rstee\AppData\Local\Temp\ipykernel_4420\1735829786.py:1: SyntaxWarning: invalid escape sequence '\R'
  calendario = pd.read_csv("C:\RS\España\Master Data Science\TFM\capstone-project-2\capstone-project-r2\capstone-project-r2\data\daily_calendar_with_events.csv")
C:\Users\rstee\AppData\Local\Temp\ipykernel_4420\1735829786.py:2: SyntaxWarning: invalid escape sequence '\R'
  precios = pd.read_csv("C:\RS\España\Master Data Science\TFM\capstone-project-2\capstone-project-r2\capstone-project-r2\data\item_prices.csv")
C:\Users\rstee\AppData\Local\Temp\ipykernel_4420\1735829786.py:3: SyntaxWarning: invalid escape sequence '\R'
  ventas = pd.read_csv("C:\RS\España\Master Data Science\TFM\cap

In [12]:
calendario.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1913 entries, 0 to 1912
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         1913 non-null   object
 1   weekday      1913 non-null   object
 2   weekday_int  1913 non-null   int64 
 3   d            1913 non-null   object
 4   event        26 non-null     object
dtypes: int64(1), object(4)
memory usage: 74.9+ KB


In [59]:
calendario.head()

,date,weekday,weekday_int,d,event
0,2011-01-29,Saturday,1,d_1,NaN
1,2011-01-30,Sunday,2,d_2,NaN
2,2011-01-31,Monday,3,d_3,NaN
3,2011-02-01,Tuesday,4,d_4,NaN
4,2011-02-02,Wednesday,5,d_5,NaN


In [92]:
precios.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6965706 entries, 0 to 6965705
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   item        object 
 1   category    object 
 2   store_code  object 
 3   yearweek    float64
 4   sell_price  float64
dtypes: float64(2), object(3)
memory usage: 265.7+ MB


In [67]:
precios.head()

,item,category,store_code,yearweek,sell_price
0,ACCESORIES_1_001,ACCESORIES,NYC_1,201328.0,12.7414
1,ACCESORIES_1_001,ACCESORIES,NYC_1,201329.0,12.7414
2,ACCESORIES_1_001,ACCESORIES,NYC_1,201330.0,10.9858
3,ACCESORIES_1_001,ACCESORIES,NYC_1,201331.0,10.9858
4,ACCESORIES_1_001,ACCESORIES,NYC_1,201332.0,10.9858


In [14]:
ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1920 entries, id to d_1913
dtypes: int64(1913), object(7)
memory usage: 446.6+ MB


In [15]:
ventas.head()


,id,item,category,department,store,store_code,region,d_1,d_2,d_3,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [37]:
suma_total_index=ventas.sum(axis=1,numeric_only=True)

In [38]:
ventas1=ventas.copy()

In [41]:
ventas1["ventas Total"]=suma_total_index
ventas1["ventas Total"].sum()

65695409

In [48]:
ventasmelt=ventas.melt(id_vars=ventas.columns[:7],value_vars=ventas.columns[7:])


In [49]:
ventasmelt["value"].sum()

65695409

In [52]:
ventasmelt[ventasmelt["value"]==0].index

Index([       0,        1,        2,        3,        4,        5,        6,
              9,       10,       11,
       ...
       58327349, 58327350, 58327351, 58327355, 58327357, 58327358, 58327363,
       58327366, 58327367, 58327369],
      dtype='int64', length=39777094)

In [53]:
ventas2=ventasmelt.drop(ventasmelt[ventasmelt["value"]==0].index)

In [56]:
ventas2.reset_index(inplace=True)

In [57]:
ventas2["value"].sum()

65695409

In [58]:
ventas2

,index,id,item,category,department,store,store_code,region,variable,value
0,7,ACCESORIES_1_008_NYC_1,ACCESORIES_1_008,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,12
1,8,ACCESORIES_1_009_NYC_1,ACCESORIES_1_009,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,2
2,14,ACCESORIES_1_015_NYC_1,ACCESORIES_1_015,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,4
3,15,ACCESORIES_1_016_NYC_1,ACCESORIES_1_016,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,5
4,21,ACCESORIES_1_022_NYC_1,ACCESORIES_1_022,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,2
...,...,...,...,...,...,...,...,...,...,...
18550271,58327361,SUPERMARKET_3_819_PHI_3,SUPERMARKET_3_819,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,2
18550272,58327362,SUPERMARKET_3_820_PHI_3,SUPERMARKET_3_820,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,1
18550273,58327364,SUPERMARKET_3_822_PHI_3,SUPERMARKET_3_822,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,1
18550274,58327365,SUPERMARKET_3_823_PHI_3,SUPERMARKET_3_823,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,1


In [62]:
ventas2.rename(columns={"variable":"d","value":"Cantidad"},inplace=True)

In [116]:
ventas3=pd.merge(ventas2,calendario2,how="left",on="d")

In [117]:
ventas3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18550276 entries, 0 to 18550275
Data columns (total 18 columns):
 #   Column       Dtype         
---  ------       -----         
 0   index        int64         
 1   id           object        
 2   item         object        
 3   category     object        
 4   department   object        
 5   store        object        
 6   store_code   object        
 7   region       object        
 8   d            object        
 9   Cantidad     int64         
 10  date         datetime64[ns]
 11  weekday      object        
 12  weekday_int  int64         
 13  event        object        
 14  week         UInt32        
 15  day          int32         
 16  month        int32         
 17  year         int32         
dtypes: UInt32(1), datetime64[ns](1), int32(3), int64(3), object(10)
memory usage: 2.2+ GB


In [118]:
ventas3.head()

,index,id,item,category,department,store,store_code,region,d,Cantidad,date,weekday,weekday_int,event,week,day,month,year
0,7,ACCESORIES_1_008_NYC_1,ACCESORIES_1_008,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,12,2011-01-29,Saturday,1,NaN,4,29,1,2011
1,8,ACCESORIES_1_009_NYC_1,ACCESORIES_1_009,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,2,2011-01-29,Saturday,1,NaN,4,29,1,2011
2,14,ACCESORIES_1_015_NYC_1,ACCESORIES_1_015,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,4,2011-01-29,Saturday,1,NaN,4,29,1,2011
3,15,ACCESORIES_1_016_NYC_1,ACCESORIES_1_016,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,5,2011-01-29,Saturday,1,NaN,4,29,1,2011
4,21,ACCESORIES_1_022_NYC_1,ACCESORIES_1_022,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,2,2011-01-29,Saturday,1,NaN,4,29,1,2011


In [96]:
precios2=precios.copy()

In [101]:
precios["yearweek"].isnull().sum()

243920

In [102]:
precios2["yearweek"]=precios2["yearweek"].astype(str)
precios2["year"]=precios2["yearweek"].str.slice(stop=4)
precios2

,item,category,store_code,yearweek,sell_price,year
0,ACCESORIES_1_001,ACCESORIES,NYC_1,201328.0,12.7414,2013
1,ACCESORIES_1_001,ACCESORIES,NYC_1,201329.0,12.7414,2013
2,ACCESORIES_1_001,ACCESORIES,NYC_1,201330.0,10.9858,2013
3,ACCESORIES_1_001,ACCESORIES,NYC_1,201331.0,10.9858,2013
4,ACCESORIES_1_001,ACCESORIES,NYC_1,201332.0,10.9858,2013
...,...,...,...,...,...,...
6965701,SUPERMARKET_3_827,SUPERMARKET,PHI_3,nan,1.2000,nan
6965702,SUPERMARKET_3_827,SUPERMARKET,PHI_3,nan,1.2000,nan
6965703,SUPERMARKET_3_827,SUPERMARKET,PHI_3,nan,1.2000,nan
6965704,SUPERMARKET_3_827,SUPERMARKET,PHI_3,nan,1.2000,nan


In [105]:
precios2["semana"]=precios2["yearweek"].str.slice(start=4,stop=6)
precios2["semana"].unique()

array(['28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38',
       '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49',
       '50', '51', '52', '00', '01', '02', '03', '04', '05', '06', '07',
       '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18',
       '19', '20', '21', '22', '23', '24', '25', '26', '27', '', '53'],
      dtype=object)

In [106]:
calendario2=calendario.copy()

In [112]:
calendario2["date"]=pd.to_datetime(calendario2["date"])

In [115]:
calendario2['week'] = calendario2['date'].dt.isocalendar().week
calendario2['day'] = calendario2['date'].dt.day
calendario2['month'] = calendario2['date'].dt.month
calendario2['year'] = calendario2['date'].dt.year
calendario2

,date,weekday,weekday_int,d,event,week,day,month,year
0,2011-01-29,Saturday,1,d_1,NaN,4,29,1,2011
1,2011-01-30,Sunday,2,d_2,NaN,4,30,1,2011
2,2011-01-31,Monday,3,d_3,NaN,5,31,1,2011
3,2011-02-01,Tuesday,4,d_4,NaN,5,1,2,2011
4,2011-02-02,Wednesday,5,d_5,NaN,5,2,2,2011
...,...,...,...,...,...,...,...,...,...
1908,2016-04-20,Wednesday,5,d_1909,NaN,16,20,4,2016
1909,2016-04-21,Thursday,6,d_1910,NaN,16,21,4,2016
1910,2016-04-22,Friday,7,d_1911,NaN,16,22,4,2016
1911,2016-04-23,Saturday,1,d_1912,NaN,16,23,4,2016
